In [1]:
import pandas as pd
import numpy as np
import os, chardet, glob
import matplotlib.pyplot as plt
import seaborn

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras import Sequential, optimizers, losses, layers

%matplotlib inline

In [4]:
import os
for dirname, _, filenames in os.walk('Traindata'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Traindata\AllTrips.csv
Traindata\AllTrips1.csv
Traindata\AllTrips2.csv
Traindata\Overview.xlsx
Traindata\readin.m
Traindata\TripA01.csv
Traindata\TripA02.csv
Traindata\TripA03.csv
Traindata\TripA04.csv
Traindata\TripA05.csv
Traindata\TripA06.csv
Traindata\TripA07.csv
Traindata\TripA08.csv
Traindata\TripA09.csv
Traindata\TripA10.csv
Traindata\TripA11.csv
Traindata\TripA12.csv
Traindata\TripA13.csv
Traindata\TripA14.csv
Traindata\TripA15.csv
Traindata\TripA16.csv
Traindata\TripA17.csv
Traindata\TripA18.csv
Traindata\TripA19.csv
Traindata\TripA20.csv
Traindata\TripA21.csv
Traindata\TripA22.csv
Traindata\TripA23.csv
Traindata\TripA24.csv
Traindata\TripA25.csv
Traindata\TripA26.csv
Traindata\TripA27.csv
Traindata\TripA28.csv
Traindata\TripA29.csv
Traindata\TripA30.csv
Traindata\TripA31.csv
Traindata\TripA32.csv
Traindata\TripB01.csv
Traindata\TripB02.csv
Traindata\TripB03.csv
Traindata\TripB04.csv
Traindata\TripB05.csv
Traindata\TripB06.csv
Traindata\TripB07.csv
Traindata\TripB08.csv
Traind

In [5]:
df_overview = pd.read_excel('Traindata/Overview.xlsx')
df_overview = df_overview.drop(['Unnamed: 13','Note'], axis=1)
df_overview = df_overview.dropna()
df_overview.rename(columns = {'Unnamed: 8':'SoC difference'}, inplace = True)
df_overview.head()

,Trip,Date,Route/Area,Weather,Battery Temperature (Start) [°C],Battery Temperature (End),Battery State of Charge (Start),Battery State of Charge (End),SoC difference,Ambient Temperature (Start) [°C],Target Cabin Temperature,Distance [km],Duration [min],Fan
0,TripA01,2019-06-25_13-21-14,Munich East,sunny,21.0,22.0,0.863,0.803,0.060,25.5,23.0,7.427690,16.820000,"Automatic, Level 1"
1,TripA02,2019-06-25_14-05-31,Munich East,sunny,23.0,26.0,0.803,0.673,0.130,32.0,23.0,23.509709,23.550000,"Automatic, Level 1"
2,TripA03,2019-06-28_10-02-15,Munich East,sunny,24.0,25.0,0.835,0.751,0.084,21.5,27.0,12.820846,11.180000,"Automatic, Level 1"
3,TripA04,2019-06-28_10-13-30,Munich East,sunny,25.0,27.0,0.751,0.667,0.084,24.0,22.0,10.727491,6.870000,"Automatic, Level 1"
4,TripA05,2019-06-28_10-20-26,Munich East,sunny,27.0,27.0,0.667,0.602,0.065,24.5,24.0,12.393223,22.776667,"Automatic, Level 1"


In [6]:
def combine_csvs():
    ''' Combine all csv datasets into a single file'''
    df_master = pd.DataFrame()
    for filename in glob.glob('Traindata/'+'*.csv'):
        df_trip = pd.read_csv(
            filename,
            sep=';',
            encoding=chardet.detect(open(filename, 'rb').read())['encoding']
        )
        df_master = pd.concat([df_master, df_trip])
        print(df_master)
        df_master.to_csv('/Traindata/AllTrips1.csv', index=False)
    return df_master


def plot_dataframe_subplots(df, nrows, ncols, figsize=(30, 24)):
    """Plot a nrows x ncols grid of the df variables"""
    measurements = list(df.columns)
    plt.figure(figsize=figsize) 
    for idx, key in enumerate(measurements):
        plt.subplot(nrows, ncols, idx+1)
        plt.plot(np.array(range(df.shape[0]))*0.1, df[key])
        plt.ylabel(key)
        plt.grid(True)
        plt.xlabel(measurements[0])
        plt.show

In [7]:
df_master = combine_csvs()

In [ ]:
df_master = df_master.iloc[: , :-2]
print(list(df_master.columns.values))

In [ ]:
plot_dataframe_subplots(df_master, nrows=12, ncols=4, figsize=(30, 48))

In [ ]:
df_soc = df_master[['SoC [%]', 'displayed SoC [%]']].dropna()

# Let us look at battery SoC degradation over vehicle exploitation
plt.figure(figsize=(30, 6), facecolor='black')
ax = plt.axes()
ax.set_facecolor("black")
ax.spines['bottom'].set_color('gray')
ax.spines['top'].set_color('gray')
ax.spines['right'].set_color('gray')
ax.spines['left'].set_color('gray')
ax.tick_params(colors='white', which='both')
plt.grid(True)

plt.plot(df_soc['SoC [%]'], color="white")
plt.plot(df_soc['displayed SoC [%]'], color="yellow")
plt.xlabel('Time [ms]', color='white')
plt.ylabel('$SoC$ [%]', color='white')
plt.legend(['real','normalized'])

In [ ]:
plt.figure(figsize=(30, 16))
seaborn.heatmap(df_master.corr(),annot=True, cbar=False, cmap='Blues', fmt='.1f')

In [ ]:
df_trip = pd.read_csv(
    '/Traindata/TripB01.csv',
    sep=';',
    encoding=chardet.detect(open(f'/Traindata/TripB01.csv', 'rb').read())['encoding']
)
# Plot results of a single trips
plot_dataframe_subplots(df_trip, nrows=12, ncols=4, figsize=(30, 48))

In [ ]:
plt.figure(figsize=(30, 16))
seaborn.heatmap(df_trip.corr(),annot=True, cbar=False, cmap='Blues', fmt='.1f')

In [ ]:
def set_negative_plot_color():
    plt.figure(figsize=(30, 6), facecolor='black')
    ax = plt.axes()
    ax.set_facecolor("black")
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.tick_params(colors='white', which='both')
    plt.grid(True)

In [ ]:
set_negative_plot_color()
plt.scatter(df_trip['Battery Current [A]'], df_trip['Battery Voltage [V]'], color="yellow", s=10)
plt.xlabel('$I_{bat}$ [A]', color='white')
plt.ylabel('$V_{bat}$ [V]', color='white');

In [ ]:
df_x_y = df_master[['Time [s]','Velocity [km/h]','Elevation [m]','Longitudinal Acceleration [m/s^2]','Regenerative Braking Signal ','Battery Voltage [V]',
                    'Battery Current [A]','Battery Temperature [°C]','Heating Power CAN [kW]','AirCon Power [kW]','SoC [%]']].dropna()

plot_dataframe_subplots(df_x_y, nrows=3, ncols=4, figsize=(30, 12))

In [ ]:
df_x_y = df_master[['Time [s]','Velocity [km/h]','Elevation [m]','Longitudinal Acceleration [m/s^2]','Regenerative Braking Signal ','Battery Voltage [V]',
                    'Battery Current [A]','Battery Temperature [°C]','Heating Power CAN [kW]','AirCon Power [kW]','SoC [%]']].dropna()

plot_dataframe_subplots(df_x_y, nrows=3, ncols=4, figsize=(30, 12))

In [ ]:
X = df_x_y.drop(['Time [s]','SoC [%]'], axis=1)
y = df_x_y[['SoC [%]']]
splitter = int(len(X)*0.7)
X_train = X.values[:splitter]
X_test = X.values[splitter:]
y_train = y.values[:splitter]
y_test = y.values[splitter:]
# Check the dataset sizes
X_train.shape, y_train.shape, X_test.shape, y_test.shape[0]

In [ ]:
def regression_matrics(y_test, y_pred):
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Root Mean Square Error: {rmse}')
    print(f'Mean Absolute Error: {mae}')


In [ ]:
def plot_comparison(y_test, y_pred, xlabel, ylabel, dark_mode=False, figsize=(30, 6)):
    if dark_mode:
        plt.figure(figsize=figsize, facecolor='black')
        ax = plt.axes()
        ax.set_facecolor("black")
        ax.spines['bottom'].set_color('gray')
        ax.spines['top'].set_color('gray')
        ax.spines['right'].set_color('gray')
        ax.spines['left'].set_color('gray')
        ax.tick_params(colors='white', which='both')
        plt.plot(y_pred, color="yellow")
        plt.plot(y_test, color="white")
        plt.legend(['Predicted', 'Measured'])
        plt.xlabel(xlabel, color='white')
        plt.ylabel(ylabel, color='white')
    else:
        plt.figure(figsize=figsize)
        plt.plot(y_pred)
        plt.plot(y_test)        
        plt.legend(['Predicted', 'Measured'])
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
    plt.grid(True)

In [ ]:
# 1. Create a model
regressor = LinearRegression()
# 2. Fit the model
regressor.fit(X_train, y_train)
# 3. Make predicitons with the model
y_pred_lr = regressor.predict(X_test)
# 4. Evaluate and visualize
print('Linear Regression:')
regression_matrics(y_test, y_pred_lr)
plot_comparison(y_test=y_test, y_pred=y_pred_lr, xlabel="Time [ms]", ylabel="$SoC$ [%]", dark_mode=False)

In [ ]:
regressor = Sequential([
    layers.Dense(100),
    layers.Dense(1),
])
# 2. Compile a model
regressor.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss=losses.mae,
                  metrics=["mae"])
# 3. Fit a model
regressor.fit(X_train, y_train, epochs=5, verbose=0)
# 4. Validate the model
y_pred_nn = regressor.predict(X_test)
# 5. Evaluate and visualize
print('Neural Network:')
regression_matrics(y_test, y_pred_nn)
plot_comparison(y_test=y_test, y_pred=y_pred_nn, xlabel="Time [ms]", ylabel="$SoC$ [%]", dark_mode=False)

In [ ]:
# 1. Create a model
regressor = Sequential([
    layers.Dense(10),
    layers.Dense(10),
    layers.Dense(1),
])
# 2. Compile a model
regressor.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss=losses.mae,
                  metrics=["mae"])
# 3. Fit a model
regressor.fit(X_train, y_train, epochs=10, verbose=0)
# 4. Validate the model
y_pred_nn = regressor.predict(X_test)
# 5. Evaluate and visualize
print('Neural Network:')
regression_matrics(y_test, y_pred_nn)
plot_comparison(y_test=y_test, y_pred=y_pred_nn, xlabel="Time [ms]", ylabel="$SoC$ [%]", dark_mode=False)

In [ ]:
# 1. Create a model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation

# Linear (Identity):

# Function: linear or no activation function.
# Example: Activation('linear')
# Sigmoid (Logistic):

# Function: S-shaped curve between 0 and 1.
# Example: Activation('sigmoid')
# Hyperbolic Tangent (Tanh):

# Function: S-shaped curve between -1 and 1.
# Example: Activation('tanh')
# Rectified Linear Unit (ReLU):

# Function: Returns the input for positive values and zero for negative values.
# Example: Activation('relu')
# Leaky Rectified Linear Unit (Leaky ReLU):

# Function: Similar to ReLU but allows a small, non-zero gradient for negative values.
# Example: Activation('LeakyReLU')
# Parametric Rectified Linear Unit (PReLU):

# Function: A variant of Leaky ReLU with learnable parameters.
# Example: PReLU()
# Exponential Linear Unit (ELU):

# Function: Similar to ReLU but allows negative values with an exponential slope.
# Example: Activation('elu')
# Scaled Exponential Linear Unit (SELU):

# Function: A self-normalizing variant of the ELU.
# Example: Activation('selu')
# Softmax:

# Function: Converts a vector of values into a probability distribution.
# Example: Activation('softmax')
# Swish:

# Function: A smooth, non-monotonic activation function.
# Example: Activation('swish')
import tensorflow as tf

def my_custom_activation(x):
    return tf.sin(x) + tf.cos(x)



regressor = Sequential([
    Dense(10),
    Activation(my_custom_activation),  # Add a ReLU activation layer after the first Dense layer
    Dense(100),
    Activation(my_custom_activation),  # Add a ReLU activation layer after the second Dense layer
    Dense(100),
    Activation(my_custom_activation),  # Add a ReLU activation layer after the third Dense layer
    Dense(1),
])

# 2. Compile a model
regressor.compile(optimizer=optimizers.SGD(learning_rate=0.01),
                  loss=losses.mae,
                  metrics=["mae"])
# 3. Fit a model
regressor.fit(X_train, y_train, epochs=20, verbose=0)
# 4. Validate the model
y_pred_nn = regressor.predict(X_test)
# 5. Evaluate and visualize
print('Neural Network:')
regression_matrics(y_test, y_pred_nn)
plot_comparison(y_test=y_test, y_pred=y_pred_nn, xlabel="Time [ms]", ylabel="$SoC$ [%]", dark_mode=False)

In [ ]:
X

In [ ]:
import torch
X=torch.tensor(X.values)
y=torch.tensor(y.values)

In [ ]:
X_train = X[:splitter]
X_test = X[splitter:]
y_train = y[:splitter]
y_test = y[splitter:]
# Check the dataset sizes
X_train.shape, y_train.shape, X_test.shape, y_test.shape[0]

In [ ]:
# from torch import nn
# from torch import Tensor
# import math
from torch import nn
sm = nn.Sigmoid()
rl = nn.ReLU()
# th = nn.Tanh()
# def sincos(input: Tensor) -> Tensor:
#     return torch.sin(input)+torch.cos(input)

# def sinc(input: Tensor) -> Tensor:
#     return torch.sin(torch.pi*input)/(torch.pi*input)


# def sinc(input: Tensor) -> Tensor:   # new 1
#     return torch.sin(torch.cos(input))
# def sinc(input: Tensor) -> Tensor:
#     return torch.tanh(2*torch.sin(input))
cl=sm

class Predictor(nn.Module):
    def __init__(self):
        super(Predictor,self).__init__()
        self.layer1=nn.Linear(in_features=9,out_features=81)
        self.layer2= cl
        self.layer3=nn.Linear(in_features=81,out_features=81)
        self.layer4=cl
        self.layer3=nn.Linear(in_features=81,out_features=9)
        self.layer4=cl
        self.layer5=nn.Linear(in_features=9,out_features=1)
    
    def  forward(self,x):
        return (self.layer5(self.layer4(self.layer3(self.layer2(self.layer1(x))))))
    
model=Predictor()
model

In [ ]:
model.state_dict()

In [ ]:
loss_fn=nn.L1Loss()
# optimizer=torch.optim.SGD(params=model.parameters(),lr=0.1)
optimizer= torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)



In [ ]:
X_train

In [ ]:
y_train

In [ ]:
torch.manual_seed(42)
epochs=100

for epoch in range(epochs):
    model.train()
    
    y_pred=model(X_train)
    
    
    loss=loss_fn(y_pred,y_train)
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
    
    model.eval()
    with torch.inference_mode():
        test_pred=model(X_test)
        
        test_loss=loss_fn(test_pred,y_test)
        
    if epoch%10==0:
        print(f"Epoch: {epoch}| Loss: {loss:.5f}| Test loss: {test_loss:.5f}")
        
        
        